# Neural Network with 2 hidden layers

### Load Modules

In [1]:
# Set the Random Seed
seed_value= 2022

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import numpy  as np
import pandas  as pd
import datetime
import random
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import tensorflow as tf
from keras.models     import Sequential, load_model
from keras.layers     import Activation, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks  import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
from tensorflow.keras.losses import MeanSquaredError

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
def reset_seeds(seed_value=2022):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
reset_seeds()

### Load Dataset - include RepRisk data

In [4]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [5]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars + reprisk_vars + reprisk_rating_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 180), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [6]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [7]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [8]:
# Model
model_name = 'NN3_esg'

def create_NN3(l_rate=0.01, l1_pen=0.0001):
    model = Sequential()
    model.add(Dense(32,
                    activation='relu',
                    input_dim=X_trai.shape[1],
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(16,
                    activation='relu',
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(8,
                    activation='relu',
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=l_rate))
    return model

In [9]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

earlyStopping  = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

model_ix = 0
for l1_pen in [0.001, 0.0005, 0.0001, 0.00005, 0.00001]:
        for l_rate in [0.01, 0.005, 0.001]:
            for ensemble in list(range(0,10)):
                print("•", model_ix, ':',  end=' ')
                    
                for FOLD in range(K_FOLDs):
                    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
                    TRAI = [x for x in YEARS if x not in VALI]
                        
                    mcp_save = ModelCheckpoint(
                        '%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD),
                        save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True
                    )

                    reset_seeds(ensemble*10000)
                        
                    NN_model = create_NN3(l_rate, l1_pen)
                        
                    history = NN_model.fit(
                        X_trai.loc[TRAI], Y_trai.loc[TRAI],
                        validation_data=(X_trai.loc[VALI], Y_trai.loc[VALI]),
                        epochs=200, verbose=0, shuffle=True, batch_size=2000,
                        callbacks=[earlyStopping, mcp_save]
                    )
                    tra_loss = np.min(history.history['loss'])
                    val_loss = np.min(history.history['val_loss'])

                    results.append({
                        'model_ix' :model_ix,
                        'l_rate'   :l_rate,
                        'l1_pen'   :l1_pen,
                        'fold'     :FOLD,
                        'tra_loss' :tra_loss,
                        'val_loss' :val_loss
                    })
                    
                    # Calculate validation predictions
                    best_model = create_NN3(l_rate, l1_pen)
                    best_model.load_weights(r'%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD));
                    Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])
                    
                    # Calculate predictions for test data, if FOLD = 0
                    if FOLD==0:
                        Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)

                model_ix += 1

• 0 : Epoch 12: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 1 : Epoch 12: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 2 : Epoch 20: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 3

Epoch 18: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 42 : Epoch 14: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 19: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 17: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 43 : Epoch 15: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 16: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 16: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 44 : Epoch 11: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 14: early stopping
2156/2156 [==============================] - 5s 2ms/

2193/2193 [==============================] - 4s 2ms/step
Epoch 51: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 52: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 84 : Epoch 32: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 48: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 54: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 46: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 85 : Epoch 80: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 80: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 86: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 75: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 86 : Epoch 41: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 37: early stop

C:\Users\rafae\AppData\Local\Temp\ipykernel_13796\2818418857.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


...

Epoch 32: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 40: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 48: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 149 : Epoch 65: early stopping
   1/1929 [..............................] - ETA: 30s

C:\Users\rafae\AppData\Local\Temp\ipykernel_13796\2818418857.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


C:\Users\rafae\AppData\Local\Temp\ipykernel_13796\2818418857.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)


Epoch 34: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 38: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 35: early stopping
2066/2066 [==============================] - 4s 2ms/step


In [10]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  NN3_esg_0  NN3_esg_1  NN3_esg_2  NN3_esg_3  \
year YM      permno                                                         
2007 2007-01 10025  -0.031894   0.007824   0.009145   0.009784   0.008453   
             10026  -0.042317   0.007824   0.009145   0.009784   0.008453   
             10042  -0.125751   0.007824   0.009145   0.009784   0.008453   
             10078  -0.080607   0.007824   0.009145   0.009784   0.008453   
             10104  -0.046341   0.007824   0.009145   0.009784   0.008453   
...                       ...        ...        ...        ...        ...   
2018 2018-12 93420   0.086508   0.010052   0.002852   0.006192   0.005177   
             93422   0.466817   0.010052   0.002852   0.006192   0.005177   
             93423   0.105036   0.010052   0.002852   0.006192   0.005177   
             93429  -0.048712   0.010052   0.002852   0.006192   0.005177   
             93436  -0.079564   0.010052   0.002852   0.006192   0.005177   

                     NN3_esg_4  NN3_esg_5  NN3_esg_6  NN3_esg_7  NN3_esg_8  \
year YM      permno                                                          
2007 2007-01 10025    0.008142   0.008171   0.009099   0.005816   0.005462   
             10026    0.008142   0.008171   0.009099   0.005816   0.005462   
             10042    0.008142   0.008171   0.009099   0.005816   0.005462   
             10078    0.008142   0.008171   0.009099   0.005816   0.005462   
             10104    0.008142   0.008171   0.009099   0.005816   0.005462   
...                        ...        ...        ...        ...        ...   
2018 2018-12 93420    0.005633   0.006116   0.008087   0.003023   0.004796   
             93422    0.005633   0.006116   0.008087   0.003023   0.004796   
             93423    0.005633   0.006116   0.008087   0.003023   0.004796   
             93429    0.005633   0.006116   0.008087   0.003023   0.004796   
             93436    0.005633   0.006116   0.008087   0.003023   0.004796   

                     ...  NN3_esg_140  NN3_esg_141  NN3_esg_142  NN3_esg_143  \
year YM      permno  ...                                                       
2007 2007-01 10025   ...     0.009843     0.016919     0.012500     0.017613   
             10026   ...     0.006599     0.008595     0.011061     0.014685   
             10042   ...    -0.021312    -0.042295    -0.019548    -0.037837   
             10078   ...     0.007604     0.008573     0.011661     0.010414   
             10104   ...     0.013146     0.012072     0.011608     0.012206   
...                  ...          ...          ...          ...          ...   
2018 2018-12 93420   ...     0.013659    -0.040602    -0.056691    -0.036125   
             93422   ...     0.006881    -0.025186    -0.049911    -0.038782   
             93423   ...     0.005396     0.010089     0.008261     0.009035   
             93429   ...     0.009862     0.012782     0.018609     0.010482   
             93436   ...    -0.009083     0.011283     0.010283     0.008415   

                     NN3_esg_144  NN3_esg_145  NN3_esg_146  NN3_esg_147  \
year YM      permno                                                       
2007 2007-01 10025      0.017197     0.014508     0.013256     0.009733   
             10026      0.011529     0.012830     0.013196     0.009547   
             10042     -0.035114    -0.029322    -0.019685     0.000925   
             10078      0.012297     0.012046     0.013162     0.011721   
             10104      0.013650     0.013626     0.013201     0.008335   
...                          ...          ...          ...          ...   
2018 2018-12 93420      0.011964    -0.006735     0.015719     0.028013   
             93422     -0.005332    -0.016939     0.004126    -0.018269   
             93423      0.004346     0.014984     0.013678     0.005960   
             93429      0.011930     0.013978     0.009772     0.006531   
             93436      0.011544     0.008846     0.010306     0.

In [11]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  NN3_esg_0  NN3_esg_1  NN3_esg_2  NN3_esg_3  \
year YM      permno                                                         
2019 2019-01 10026   0.004225   0.007824   0.009145   0.009784   0.008453   
             10104   0.036026   0.007824   0.009145   0.009784   0.008453   
             10107   0.075381   0.007824   0.009145   0.009784   0.008453   
             10138   0.072777   0.007824   0.009145   0.009784   0.008452   
             10145   0.076596   0.007824   0.009145   0.009784   0.008453   
...                       ...        ...        ...        ...        ...   
2021 2021-12 93304  -0.096386   0.007824   0.009145   0.009784   0.008453   
             93373  -0.019481   0.007824   0.009145   0.009784   0.008452   
             93374  -0.047552   0.007824   0.009145   0.009784   0.008453   
             93423  -0.072569   0.007824   0.009145   0.009784   0.008452   
             93436  -0.113609   0.007824   0.009145   0.009784   0.008452   

                     NN3_esg_4  NN3_esg_5  NN3_esg_6  NN3_esg_7  NN3_esg_8  \
year YM      permno                                                          
2019 2019-01 10026    0.008142   0.008171   0.009099   0.005816   0.005462   
             10104    0.008142   0.008171   0.009099   0.005816   0.005462   
             10107    0.008142   0.008171   0.009099   0.005816   0.005462   
             10138    0.008142   0.008171   0.009099   0.005816   0.005462   
             10145    0.008142   0.008171   0.009099   0.005816   0.005462   
...                        ...        ...        ...        ...        ...   
2021 2021-12 93304    0.008142   0.008171   0.009099   0.005816   0.005462   
             93373    0.008142   0.008171   0.009099   0.005816   0.005462   
             93374    0.008142   0.008171   0.009099   0.005816   0.005462   
             93423    0.008142   0.008171   0.009099   0.005816   0.005462   
             93436    0.008142   0.008171   0.009099   0.005816   0.005462   

                     ...  NN3_esg_140  NN3_esg_141  NN3_esg_142  NN3_esg_143  \
year YM      permno  ...                                                       
2019 2019-01 10026   ...     0.007414     0.009414     0.011061     0.012436   
             10104   ...     0.008076     0.001799     0.010744     0.007984   
             10107   ...     0.008269     0.010013     0.011317     0.012226   
             10138   ...     0.005698     0.007506     0.010189     0.011110   
             10145   ...     0.007069     0.007874     0.010781     0.010589   
...                  ...          ...          ...          ...          ...   
2021 2021-12 93304   ...     0.001876     0.003901     0.008231     0.004674   
             93373   ...    -0.010876     0.001048    -0.006662     0.003841   
             93374   ...     0.010761     0.009292     0.010522     0.007461   
             93423   ...     0.011881     0.007459     0.010440     0.011738   
             93436   ...     0.005455     0.007521     0.009408     0.003200   

                     NN3_esg_144  NN3_esg_145  NN3_esg_146  NN3_esg_147  \
year YM      permno                                                       
2019 2019-01 10026      0.010060     0.011357     0.013178     0.008921   
             10104      0.011891     0.010220     0.013134     0.005092   
             10107      0.010787     0.010799     0.013206     0.008671   
             10138      0.008991     0.011291     0.011560     0.008115   
             10145      0.009455     0.009369     0.012515     0.008379   
...                          ...          ...          ...          ...   
2021 2021-12 93304      0.008164     0.011470     0.010711     0.005544   
             93373     -0.018691    -0.028979    -0.001320     0.005135   
             93374      0.011239     0.012155     0.011484     0.009105   
             93423      0.010225     0.010432     0.009582     0.009410   
             93436      0.008383     0.008709     0.009001     0.

In [12]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'l_rate', 'l1_pen']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['tra_loss', 'val_loss']].head(20)) * 100

,,,tra_loss,val_loss
model_ix,l_rate,l1_pen,,
119,0.001,0.00005,12.978966,12.945000
115,0.001,0.00005,12.979967,12.945942
110,0.001,0.00005,12.986546,12.952208
117,0.001,0.00005,12.984312,12.952738
88,0.001,0.00010,12.991347,12.953602
112,0.001,0.00005,12.985433,12.954408
89,0.001,0.00010,12.991492,12.955815
85,0.001,0.00010,12.990758,12.956764
116,0.001,0.00005,12.987198,12.958856


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [13]:
# Model
model_name = 'NN3_esg'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_l_rate = list(pd.unique(table['l_rate']))
list_l1_pen = list(pd.unique(table['l1_pen']))

In [14]:
# Loop over hyperparameter combinations to get ensemble predictions for each hyperparameter combination
val_results = []
hyper_comb = 0
for l_rate in list_l_rate:
    for l1_pen in list_l1_pen:
        # Get list of relevant model_ix
        list_model_ix = table[(table['l_rate'] == l_rate) & (table['l1_pen'] == l1_pen)]
        list_model_ix = list(list_model_ix['model_ix'])
        # Get list of column names
        list_col_names = []
        for ele in list_model_ix:
            list_col_names.append('%s_%d'%(model_name, ele))
        # Select relevant returns and average predictions
        val_ret = val[list_col_names]
        val_ret = val_ret.mean(axis=1)
        # Append ensemble return prediction to true returns
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'l_rate'    :l_rate,
            'l1_pen'    :l1_pen,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1
        
# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))

In [15]:
# Select best hyperparemeters (max. SR_LS)
val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['l_rate', 'l1_pen']).sort_values(['SR_LS'], ascending=False)
l_rate_opt, l1_pen_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal l_rate:', l_rate_opt)
print('Optimal l1_pen:', l1_pen_opt)
val_results

Optimal l_rate: 0.01
Optimal l1_pen: 1e-05


Unnamed: 0  hyper_comb     SR_Q1     SR_Q5     SR_LS  \
l_rate l1_pen                                                          
0.010  0.00001           4           4  0.013974  0.577002  0.746016   
0.001  0.00001          14          14  0.085011  0.430992  0.570476   
0.005  0.00100           5           5  0.241002  0.471465  0.521529   
       0.00001           9           9  0.068871  0.410000  0.410042   
0.010  0.00010           2           2  0.297167  0.414613  0.382376   
0.001  0.00100          10          10  0.312678  0.390878  0.317877   
       0.00005          13          13  0.172231  0.521262  0.247611   
0.005  0.00050           6           6  0.296397  0.368865  0.153025   
0.001  0.00050          11          11  0.379683  0.290386 -0.022467   
0.005  0.00005           8           8  0.273322  0.369735 -0.027533   
0.010  0.00005           3           3  0.270060  0.363338 -0.045085   
0.005  0.00010           7           7  0.369464  0.325028 -0.162121   
0.010  0.00050           1           1  0.428230  0.364614 -0.290251   
0.001  0.00010          12          12  0.368687  0.277010 -0.328827   
0.010  0.00100           0           0  0.428700  0.322872 -0.337514   

                 Mean_Q1   Mean_Q5   Mean_LS  
l_rate l1_pen                                 
0.010  0.00001  0.000336  0.009240  0.008904  
0.001  0.00001  0.001914  0.007669  0.005755  
0.005  0.00100  0.004472  0.007938  0.003467  
       0.00001  0.001635  0.006460  0.004825  
0.010  0.00010  0.004954  0.007826  0.002871  
0.001  0.00100  0.005354  0.007056  0.001703  
       0.00005  0.004105  0.007423  0.003318  
0.005  0.00050  0.005397  0.006624  0.001226  
0.001  0.00050  0.005870  0.005701 -0.000169  
0.005  0.00005  0.005971  0.005646 -0.000325  
0.010  0.00005  0.006161  0.005691 -0.000469  
0.005  0.00010  0.006535  0.005439 -0.001096  
0.010  0.00050  0.007712  0.006060 -0.001652  
0.001  0.00010  0.006597  0.004905 -0.001692  
0.010  0.00100  0.007479  0.005607 -0.001872

## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [16]:
# Get list of model_ix's with best hyperparameter combinations and make ensemble predictions
list_model_ix = table[(table['l_rate'] == l_rate_opt) & (table['l1_pen'] == l1_pen_opt)]
list_model_ix = list(list_model_ix['model_ix'])

# Get list of column names
list_col_names = []
for ele in list_model_ix:
    list_col_names.append('%s_%d'%(model_name, ele))
    
# Select relevant returns and average predictions
test_ret = test[list_col_names].copy()
test_ret['y_pred'] = test_ret.mean(axis=1)

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)